In [ ]:
import numpy as np
import scipy.stats as stats
import scipy.signal as signal
from scipy.ndimage import median_filter
import statsmodels.tsa.stattools as stattools
import os
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

MIN_WINDOW_SEC = 2  

def extract_features(xyz, sample_rate=100):
    '''Extract commonly used HAR time-series features. xyz is a window of shape (N,3)'''
    
    if np.isnan(xyz).any():
        return {}
    
    if len(xyz) <= MIN_WINDOW_SEC * sample_rate:
        return {}
    
    feats = {}
    
    v = np.linalg.norm(xyz, axis=1)
    v = median_filter(v, size=5, mode='nearest')
    v = v - 1  # detrend: "remove gravity"
    v = np.clip(v, -2, 2)  # clip abnormaly high values
    
    # Moments features
    feats.update(moments_features(v, sample_rate))
    
    # Quantile features
    feats.update(quantile_features(v, sample_rate))
    
    # Autocorrelation features
    feats.update(autocorr_features(v, sample_rate))
    
    # Spectral features
    feats.update(spectral_features(v, sample_rate))
    
    # FFT features
    feats.update(fft_features(v, sample_rate))
    
    # Peak features
    feats.update(peaks_features(v, sample_rate))
    
    return feats


def moments_features(v, sample_rate=None):
    """Moments"""
    avg = np.mean(v)
    std = np.std(v)
    if std > .01:
        skew = np.nan_to_num(stats.skew(v))
        kurt = np.nan_to_num(stats.kurtosis(v))
    else:
        skew = kurt = 0
    feats = {
        'avg': avg,
        'std': std,
        'skew': skew,
        'kurt': kurt,
    }
    return feats


def quantile_features(v, sample_rate=None):
    """Quantiles (min, 25th, med, 75th, max)"""
    feats = {}
    feats['min'], feats['q25'], feats['med'], feats['q75'], feats['max'] = np.quantile(v, (0, .25, .5, .75, 1))
    return feats


def autocorr_features(v, sample_rate):
    """Autocorrelation features"""
    
    with np.errstate(divide='ignore', invalid='ignore'):  # ignore invalid div warnings
        u = np.nan_to_num(stattools.acf(v, nlags=2 * sample_rate))
    
    peaks, _ = signal.find_peaks(u, prominence=.1)
    if len(peaks) > 0:
        acf_1st_max_loc = peaks[0]
        acf_1st_max = u[acf_1st_max_loc]
        acf_1st_max_loc /= sample_rate  # in secs
    else:
        acf_1st_max = acf_1st_max_loc = 0.0
    
    valleys, _ = signal.find_peaks(-u, prominence=.1)
    if len(valleys) > 0:
        acf_1st_min_loc = valleys[0]
        acf_1st_min = u[acf_1st_min_loc]
        acf_1st_min_loc /= sample_rate  # in secs
    else:
        acf_1st_min = acf_1st_min_loc = 0.0
    
    acf_zeros = np.sum(np.diff(np.signbit(u)))
    
    feats = {
        'acf_1st_max': acf_1st_max,
        'acf_1st_max_loc': acf_1st_max_loc,
        'acf_1st_min': acf_1st_min,
        'acf_1st_min_loc': acf_1st_min_loc,
        'acf_zeros': acf_zeros,
    }
    
    return feats


def spectral_features(v, sample_rate):
    """Spectral entropy, average power, dominant frequencies"""
    
    feats = {}
    
    freqs, powers = signal.periodogram(v, fs=sample_rate, detrend='constant', scaling='density')
    powers /= (len(v) / sample_rate)    # unit/sec
    
    feats['pentropy'] = stats.entropy(powers[powers > 0])
    feats['power'] = np.sum(powers)
    
    peaks, _ = signal.find_peaks(powers)
    peak_powers = powers[peaks]
    peak_freqs = freqs[peaks]
    peak_ranks = np.argsort(peak_powers)[::-1]
    
    TOPN = 3
    feats.update({f"f{i + 1}": 0 for i in range(TOPN)})
    feats.update({f"p{i + 1}": 0 for i in range(TOPN)})
    for i, j in enumerate(peak_ranks[:TOPN]):
        feats[f"f{i + 1}"] = peak_freqs[j]
        feats[f"p{i + 1}"] = peak_powers[j]
    
    return feats


def fft_features(v, sample_rate, nfreqs=5):
    """Power of frequencies 0Hz, 1Hz, 2Hz, ... using Welch's method"""
    
    _, powers = signal.welch(
        v, fs=sample_rate,
        nperseg=sample_rate,
        noverlap=sample_rate // 2,
        detrend='constant',
        scaling='density',
        average='median'
    )
    
    feats = {f"fft{i}": powers[i] for i in range(nfreqs + 1)}
    
    return feats


def peaks_features(v, sample_rate):
    """Features of the signal peaks"""
    
    feats = {}
    u = butterfilt(v, 5, fs=sample_rate)  # lowpass 5Hz
    peaks, peak_props = signal.find_peaks(u, distance=0.2 * sample_rate, prominence=0.25)
    feats['npeaks'] = len(peaks) / (len(v) / sample_rate)  # peaks/sec
    if len(peak_props['prominences']) > 0:
        feats['peaks_avg_promin'] = np.mean(peak_props['prominences'])
        feats['peaks_min_promin'] = np.min(peak_props['prominences'])
        feats['peaks_max_promin'] = np.max(peak_props['prominences'])
    else:
        feats['peaks_avg_promin'] = feats['peaks_min_promin'] = feats['peaks_max_promin'] = 0
    
    return feats


def butterfilt(x, cutoffs, fs, order=4, axis=0):
    """Butterworth filter"""
    nyq = 0.5 * fs
    if isinstance(cutoffs, tuple):
        hicut, lowcut = cutoffs
        if hicut > 0:
            btype = 'bandpass'
            Wn = (hicut / nyq, lowcut / nyq)
        else:
            btype = 'low'
            Wn = lowcut / nyq
    else:
        btype = 'low'
        Wn = cutoffs / nyq
    sos = signal.butter(order, Wn, btype=btype, analog=False, output='sos')
    y = signal.sosfiltfilt(sos, x, axis=axis)
    return y


def get_feature_names():
    """Get the list of feature names in consistent order"""
    # Create a dummy window to extract feature names
    dummy_window = np.random.randn(500, 3) * 0.1
    feats = extract_features(dummy_window, 100)
    return list(feats.keys())


def extract_features_batch(X, sample_rate=100, batch_size=1000):
    """
    Extract features from all windows in X
    
    Parameters:
    -----------
    X : numpy array of shape (n_samples, n_timesteps, 3)
        The input accelerometer data
    sample_rate : int
        Sampling rate in Hz (default 100)
    batch_size : int
        Number of windows to process at once for progress updates
    
    Returns:
    --------
    X_features : numpy array of shape (n_samples, 32)
        Extracted features for all windows
    feature_names : list
        Names of the features in order
    """
    
    n_samples = X.shape[0]
    
    # Get feature names from a dummy extraction
    feature_names = get_feature_names()
    n_features = len(feature_names)
    
    print(f"Extracting {n_features} features from {n_samples} windows...")
    print(f"Features: {feature_names}")
    
    # Initialize feature matrix
    X_features = np.zeros((n_samples, n_features), dtype=np.float32)
    
    # Track failed extractions
    failed_indices = []
    
    # Process in batches for progress tracking
    for batch_start in tqdm(range(0, n_samples, batch_size), desc="Processing batches"):
        batch_end = min(batch_start + batch_size, n_samples)
        
        for i in range(batch_start, batch_end):
            try:
                # Extract features for this window
                features = extract_features(X[i], sample_rate)
                
                if features:  # If extraction was successful
                    # Fill in the feature values in consistent order
                    for j, feat_name in enumerate(feature_names):
                        X_features[i, j] = features.get(feat_name, 0.0)
                else:
                    # Mark as failed extraction
                    failed_indices.append(i)
                    
            except Exception as e:
                print(f"\nError processing window {i}: {e}")
                failed_indices.append(i)
    
    if failed_indices:
        print(f"\nWarning: {len(failed_indices)} windows failed feature extraction")
        print(f"Failed indices: {failed_indices[:10]}..." if len(failed_indices) > 10 else f"Failed indices: {failed_indices}")
    
    print(f"\nFeature extraction complete!")
    print(f"Output shape: {X_features.shape}")
    print(f"Features extracted: {feature_names}")
    
    # Basic statistics
    print(f"\nFeature statistics:")
    print(f"  Min values: {np.min(X_features, axis=0)[:5]}...")
    print(f"  Max values: {np.max(X_features, axis=0)[:5]}...")
    print(f"  Mean values: {np.mean(X_features, axis=0)[:5]}...")
    print(f"  NaN count: {np.isnan(X_features).sum()}")
    print(f"  Inf count: {np.isinf(X_features).sum()}")
    
    return X_features, feature_names


def load_prepared_data(data_dir='preprocessed', schema='WillettsSpecific2018'):
    """Load your preprocessed data"""
    print(f"Loading prepared data from: {data_dir}")
    
    X = np.load(os.path.join(data_dir, 'X.npy'))
    Y = np.load(os.path.join(data_dir, f'Y_{schema}.npy'), allow_pickle=True)
    T = np.load(os.path.join(data_dir, 'T.npy'), allow_pickle=True)
    P = np.load(os.path.join(data_dir, 'P.npy'), allow_pickle=True)
    
    print(f"\nLoaded data:")
    print(f"  X shape: {X.shape}")
    print(f"  Y shape: {Y.shape}")
    print(f"  Number of participants: {len(np.unique(P))}")
    
    return X, Y, T, P


def save_features(X_features, feature_names, save_dir='preprocessed'):
    """Save extracted features and feature names"""
    
    # Create directory if it doesn't exist
    os.makedirs(save_dir, exist_ok=True)
    
    # Save features
    features_path = os.path.join(save_dir, 'X_features.npy')
    np.save(features_path, X_features)
    print(f"\nSaved features to: {features_path}")
    
    # Save feature names
    names_path = os.path.join(save_dir, 'feature_names.npy')
    np.save(names_path, np.array(feature_names))
    print(f"Saved feature names to: {names_path}")
    
    return features_path, names_path


# Main execution
if __name__ == "__main__":
    
    # Load your data
    X, Y, T, P = load_prepared_data(schema='WillettsSpecific2018')
    
    # Extract features from all windows
    # Assuming 100Hz sample rate (1000 timesteps = 10 seconds at 100Hz)
    X_features, feature_names = extract_features_batch(X, sample_rate=100, batch_size=1000)
    
    # Save the features
    save_features(X_features, feature_names)
    
    print("\n" + "="*50)
    print("Feature extraction completed successfully!")
    print(f"Original data shape: {X.shape}")
    print(f"Feature matrix shape: {X_features.shape}")
    print(f"Ready for ML model training!")
    print("="*50)

Loading prepared data from: preprocessed

Loaded data:
  X shape: (1398022, 1000, 3)
  Y shape: (1398022,)
  Number of participants: 151
Extracting 32 features from 1398022 windows...
Features: ['avg', 'std', 'skew', 'kurt', 'min', 'q25', 'med', 'q75', 'max', 'acf_1st_max', 'acf_1st_max_loc', 'acf_1st_min', 'acf_1st_min_loc', 'acf_zeros', 'pentropy', 'power', 'f1', 'f2', 'f3', 'p1', 'p2', 'p3', 'fft0', 'fft1', 'fft2', 'fft3', 'fft4', 'fft5', 'npeaks', 'peaks_avg_promin', 'peaks_min_promin', 'peaks_max_promin']


Processing batches: 100%|██████████| 1399/1399 [30:12<00:00,  1.30s/it]



Feature extraction complete!
Output shape: (1398022, 32)
Features extracted: ['avg', 'std', 'skew', 'kurt', 'min', 'q25', 'med', 'q75', 'max', 'acf_1st_max', 'acf_1st_max_loc', 'acf_1st_min', 'acf_1st_min_loc', 'acf_zeros', 'pentropy', 'power', 'f1', 'f2', 'f3', 'p1', 'p2', 'p3', 'fft0', 'fft1', 'fft2', 'fft3', 'fft4', 'fft5', 'npeaks', 'peaks_avg_promin', 'peaks_min_promin', 'peaks_max_promin']

Feature statistics:
  Min values: [ -0.1642207    0.         -16.381773    -1.9261225   -0.98326266]...
  Max values: [1.7581493e+00 1.0460494e+00 2.8646334e+01 8.7671014e+02 6.9947183e-01]...
  Mean values: [ 0.01215498  0.06573144  0.53021324  5.2717385  -0.1811054 ]...
  NaN count: 0
  Inf count: 0

Saved features to: preprocessed/X_features.npy
Saved feature names to: preprocessed/feature_names.npy

Feature extraction completed successfully!
Original data shape: (1398022, 1000, 3)
Feature matrix shape: (1398022, 32)
Ready for ML model training!


In [4]:
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.preprocessing import LabelEncoder
import os
import warnings
warnings.filterwarnings('ignore')

def load_data_and_features(data_dir='preprocessed', schema='WillettsSpecific2018'):
    """Load features and labels"""
    print("Loading data...")
    
    # Load features
    X_features = np.load(os.path.join(data_dir, 'X_features.npy'))
    
    # Load labels
    Y = np.load(os.path.join(data_dir, f'Y_{schema}.npy'), allow_pickle=True)
    
    print(f"Features shape: {X_features.shape}")
    print(f"Labels shape: {Y.shape}")
    print(f"Label type: {Y.dtype}")
    print(f"Number of unique classes: {len(np.unique(Y))}")
    print(f"Classes: {np.unique(Y)}")
    
    return X_features, Y

def train_xgboost_model(X_features, Y, test_size=0.2, random_state=42):
    """Train a simple XGBoost model"""
    
    # Handle 'nan' labels - remove them
    print("\nHandling labels...")
    valid_indices = Y != 'nan'
    X_features = X_features[valid_indices]
    Y = Y[valid_indices]
    print(f"Removed {(~valid_indices).sum()} samples with 'nan' labels")
    print(f"Remaining samples: {len(Y)}")
    
    # Encode string labels to integers
    label_encoder = LabelEncoder()
    Y_encoded = label_encoder.fit_transform(Y)
    
    print(f"\nLabel mapping:")
    for i, label in enumerate(label_encoder.classes_):
        print(f"  {i}: {label}")
    
    # Train test split
    print("\nSplitting data...")
    X_train, X_test, y_train, y_test = train_test_split(
        X_features, Y_encoded, 
        test_size=test_size, 
        random_state=random_state,
        stratify=Y_encoded  # Maintain class distribution
    )
    
    print(f"Training set: {X_train.shape}")
    print(f"Test set: {X_test.shape}")
    
    # Initialize XGBoost classifier with basic parameters
    print("\nTraining XGBoost model...")
    model = xgb.XGBClassifier(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        objective='multi:softprob',
        n_jobs=-1,
        random_state=random_state
    )
    
    # Train the model
    model.fit(
        X_train, y_train,
        eval_set=[(X_test, y_test)],
        verbose=False
    )
    
    print("Training complete!")
    
    # Make predictions
    print("\nMaking predictions...")
    y_pred = model.predict(X_test)
    
    # Calculate metrics
    print("\nCalculating metrics...")
    accuracy = accuracy_score(y_test, y_pred)
    
    # Calculate f1, precision, recall with macro average for multi-class
    f1 = f1_score(y_test, y_pred, average='macro')
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    
    # Also calculate weighted averages (weighted by support)
    f1_weighted = f1_score(y_test, y_pred, average='weighted')
    precision_weighted = precision_score(y_test, y_pred, average='weighted')
    recall_weighted = recall_score(y_test, y_pred, average='weighted')
    
    # Print results
    print("\n" + "="*50)
    print("MODEL PERFORMANCE")
    print("="*50)
    print(f"Accuracy:  {accuracy:.4f}")
    print("\nMacro Averages (treats all classes equally):")
    print(f"F1 Score:  {f1:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print("\nWeighted Averages (weighted by class frequency):")
    print(f"F1 Score:  {f1_weighted:.4f}")
    print(f"Precision: {precision_weighted:.4f}")
    print(f"Recall:    {recall_weighted:.4f}")
    print("="*50)
    
    # Per-class performance
    print("\nPer-class Performance:")
    for i, label in enumerate(label_encoder.classes_):
        mask = y_test == i
        if mask.sum() > 0:
            class_acc = accuracy_score(y_test[mask], y_pred[mask])
            print(f"  {label:20s}: {class_acc:.3f}")
    
    return model, label_encoder, {
        'accuracy': accuracy,
        'f1_macro': f1,
        'precision_macro': precision,
        'recall_macro': recall,
        'f1_weighted': f1_weighted,
        'precision_weighted': precision_weighted,
        'recall_weighted': recall_weighted
    }

# Main execution
if __name__ == "__main__":
    
    # Load features and labels
    X_features, Y = load_data_and_features()
    
    # Train model and get metrics
    model, label_encoder, metrics = train_xgboost_model(X_features, Y)
    
    print("\nDone! Model trained and evaluated.")
    print(f"Test set accuracy: {metrics['accuracy']:.2%}")

Loading data...
Features shape: (1398022, 32)
Labels shape: (1398022,)
Label type: <U32
Number of unique classes: 11
Classes: ['bicycling' 'household-chores' 'manual-work' 'mixed-activity' 'nan'
 'sitting' 'sleep' 'sports' 'standing' 'vehicle' 'walking']

Handling labels...
Removed 475823 samples with 'nan' labels
Remaining samples: 922199

Label mapping:
  0: bicycling
  1: household-chores
  2: manual-work
  3: mixed-activity
  4: sitting
  5: sleep
  6: sports
  7: standing
  8: vehicle
  9: walking

Splitting data...
Training set: (737759, 32)
Test set: (184440, 32)

Training XGBoost model...
Training complete!

Making predictions...

Calculating metrics...

MODEL PERFORMANCE
Accuracy:  0.7161

Macro Averages (treats all classes equally):
F1 Score:  0.4636
Precision: 0.6359
Recall:    0.4384

Weighted Averages (weighted by class frequency):
F1 Score:  0.6936
Precision: 0.7117
Recall:    0.7161

Per-class Performance:
  bicycling           : 0.717
  household-chores    : 0.560
  man